In [1]:
# coding=utf-8
import qiskit.qasm3
from expr_optimal import schedule_optimal
from expr_greedy import schedule_greedy
from expr_swap import schedule_swap
from chiplet import QPU
from dag import Circuit_DAG
from qiskit import QuantumCircuit
import qiskit

# 算法类型 bv / qaoa / qft / vqa
# 量子比特数 20, 40, 60, 80, 100, 120, 140, 160, 180, 200

circuit_type = ["bv", "qaoa", "qft", "vqa"]
qubit_num = [100]
chiplet_type = ["square"]

chiplet_size = [
    {"square": 8, "hexagon": 5, "heavy_square": 5, "heavy_hexagon": 3},
    {"square": 8, "hexagon": 5, "heavy_square": 5, "heavy_hexagon": 3},
    {"square": 8, "hexagon": 5, "heavy_square": 5, "heavy_hexagon": 3},
    {"square": 8, "hexagon": 5, "heavy_square": 5, "heavy_hexagon": 3},
    {"square": 8, "hexagon": 5, "heavy_square": 5, "heavy_hexagon": 3},
]

chiplet_array = [
    {"square": [3, 3], "hexagon": [3, 3],
        "heavy_square": [3, 3], "heavy_hexagon": [3, 3]},
    {"square": [3, 3], "hexagon": [3, 3],
        "heavy_square": [3, 3], "heavy_hexagon": [3, 3]},
    {"square": [3, 3], "hexagon": [3, 3],
        "heavy_square": [3, 3], "heavy_hexagon": [3, 3]},
    {"square": [3, 3], "hexagon": [3, 3],
        "heavy_square": [3, 3], "heavy_hexagon": [3, 3]},
    {"square": [3, 3], "hexagon": [3, 3],
        "heavy_square": [3, 3], "heavy_hexagon": [3, 3]},
]

def task(num_idx, num_item, circuit_type_item, chiplet_type_item):
    qasm_file = f"circuits/{circuit_type_item}_{num_item}.qasm" # QASM文件
    ccc = qiskit.qasm3.load(qasm_file)
    circuit_depth = ccc.depth()
    print(f"{circuit_type_item} {num_item} depth: {circuit_depth}")
    circuit_dag = Circuit_DAG(qasm_file) # 电路的DAG图
    inner_chip_size = chiplet_size[num_idx][chiplet_type_item] # Chip内大小
    inter_chip_array = chiplet_array[num_idx][chiplet_type_item] # 芯片矩阵大小
    total_count = {
        "swap": [],
        "greedy": [],
        "optimal": [],
    }
    qpu = QPU(chip_type=chiplet_type_item, inner_chip_size=inner_chip_size, inter_chip_array=inter_chip_array,
              circuit_dag=circuit_dag, just_one_map=(circuit_type == "vqa" or circuit_type == "qft"))
    for func_idx, func in enumerate([schedule_swap, schedule_greedy, schedule_optimal]):
        max_depth, ancilla_num = func(qpu)
        arr = ["swap", "greedy", "optimal"]
        total_count[arr[func_idx]].append(max_depth)
        total_count[arr[func_idx]].append(ancilla_num)

for num_idx, num_item in enumerate(qubit_num):  # 100
    for circuit_type_item in circuit_type:  # "bv", "qaoa", "qft", "vqa"
        for chiplet_type_item in chiplet_type:  # "square"
            task(num_idx, num_item, circuit_type_item, chiplet_type_item)

bv 100 depth: 49
time_swap:  0.013237476348876953
time_greedy:  0.012894630432128906
time_optimal:  0.2947506904602051
qaoa 100 depth: 498
time_swap:  11.750573635101318
time_greedy:  11.663041114807129
time_optimal:  29.910369237523177
qft 100 depth: 690
time_swap:  28.85214591026306
time_greedy:  31.84979510307312
time_optimal:  84.70282578878239
vqa 100 depth: 197
time_swap:  10.838719606399536
time_greedy:  11.005696535110474
time_optimal:  30.9478698837135


In [2]:
depths = [49, 4950, 9900, 4950]
for idx, i in enumerate([[0.013237476348876953,
0.012894630432128906,
0.2947506904602051,],
[11.750573635101318,
11.663041114807129,
29.910369237523177,],
[28.85214591026306,
31.84979510307312,
84.70282578878239,],
[10.838719606399536,
11.005696535110474,
30.9478698837135,]]):
    for j in i:
        print(j / depths[idx])

0.00027015257854850924
0.00026315572310467157
0.006015320213473573
0.0023738532596164277
0.0023561699221832585
0.006042498835863268
0.002914358172753845
0.0032171510205124365
0.008555840988765898
0.002189640324525159
0.0022233730373960553
0.006252094926002727
